In [11]:
import pandas as pd
import numpy as np

r_cols = ['userId', 'movieId', 'rating']
ratings = pd.read_csv('../ml-latest-small/ratings.csv', sep=',', names=r_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)
#ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', names=r_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)
#ratings = pd.read_csv('ml-10M100K/ratings.dat', sep='::', names=r_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)

m_cols = ['movieId', 'title','genres']
movies = pd.read_csv('../ml-latest-small/movies.csv', sep=',', names=m_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)
#movies = pd.read_csv('ml-1m/movies.dat', sep='::', names=m_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)
#movies = pd.read_csv('ml-10M100K/movies.dat', sep='::', names=m_cols, usecols=range(3), encoding="ISO-8859-1", header=None, skiprows=1)

movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
ratings = pd.merge(movies, ratings)

ratings['rating'] = ratings['rating'].map(float)

ratings.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",7,3.0
1,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",9,4.0
2,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",13,5.0
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",15,2.0
4,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",19,3.0


In [22]:
movieRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating', aggfunc='first',fill_value=float('nan'))
#movieRatings.head()
movieRatings['Star Wars: Episode IV - A New Hope (1977)']

userId
1      NaN
2      NaN
3      NaN
4      5.0
5      NaN
6      NaN
7      5.0
8      3.5
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     5.0
16     NaN
17     3.5
18     3.0
19     4.0
20     1.5
21     3.0
22     4.0
23     4.5
24     NaN
25     4.0
26     3.0
27     NaN
28     NaN
29     NaN
30     4.0
      ... 
642    NaN
643    5.0
644    NaN
645    NaN
646    5.0
647    5.0
648    NaN
649    NaN
650    NaN
651    5.0
652    NaN
653    NaN
654    5.0
655    2.5
656    NaN
657    NaN
658    NaN
659    5.0
660    4.5
661    NaN
662    NaN
663    NaN
664    4.5
665    4.0
666    NaN
667    NaN
668    NaN
669    5.0
670    NaN
671    5.0
Name: Star Wars: Episode IV - A New Hope (1977), Length: 671, dtype: float64

In [13]:
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})

popularMovies = movieStats['rating']['size'] >= 80
movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                                     size      mean
title                                                              
Godfather, The (1972)                               200.0  4.487500
Shawshank Redemption, The (1994)                    311.0  4.487138
Godfather: Part II, The (1974)                      135.0  4.385185
Usual Suspects, The (1995)                          201.0  4.370647
Rear Window (1954)                                   92.0  4.315217
Schindler's List (1993)                             244.0  4.303279
North by Northwest (1959)                            87.0  4.270115
One Flew Over the Cuckoo's Nest (1975)              144.0  4.256944
Fargo (1996)                                        224.0  4.256696
Pulp Fiction (1994)                                 324.0  4.256173
American Beauty (1999)                              220.0  4.236364
Dark Knight, The (2008)                             121.0  4.235537
Casablanca (1942)                                   117.0  4.235043
Star Wars: Episode V - The Empire Strikes Back ...  234.0  4.232906
Memento (2000)                                      132.0  4.227273

In [20]:
import math

def cosine_sim(A,B):
    return (A * B).sum() / math.sqrt((A*A).sum() * (B*B).sum())

d = pd.DataFrame({'col1': [1, float('nan'), 3], 'col2': [float('nan'), 3, 1]})
cosine_sim(d['col1'], d['col2'])

0.29999999999999999

In [21]:
starWarsRatings = movieRatings['Star Wars: Episode IV - A New Hope (1977)'].map(float)
#starWarsRatings = movieRatings['Star Wars: Episode VII - The Force Awakens (2015)'].map(float)
similarMovies = movieRatings.apply(lambda x: cosine_sim(starWarsRatings, x))
#similarMovies = movieRatings.corrwith(starWarsRatings)
similarMovies = similarMovies.dropna()
#similarMovies.head()
similarMovies = similarMovies.sort_values(ascending=False)

df = pd.DataFrame(similarMovies, columns=['similarity'])
df.head()

AttributeError: 'numpy.float64' object has no attribute 'map'